# Advanced usage of HiPS and MOCs
## Exploring large catalogs within non-trivial spatial coverage, defined by brightness cuts and/or the availability of additional data sets
Originally by 

**Caroline Bot and Thomas Boch, CDS**

as published on http://www.euro-vo.org/?q=science/scientific-tutorials
converted to jupyter notebook by 

**Katharina Lutz, Thomas Boch, Matthieu Baumann, CDS**

This tutorial was first presented at "Detecting the unexpected, Discovery in the Era of Astronomically Big Data" Space Telsecope Science Instite, February 27 - March 2, 2017. It was updated to Aladin v10 in June 2017 and to Gaia DR2 in June 2018. 

This is a hands-on tutorial demonstrating an advanced useage of Hierarchical Progressive Suerveys (HiPS) and Multi-Order Coverage (MOC) maps in Aladin. Using this tutorial you will learn to handle a problem like: 
>"I have a set of images, I would like to select regions in my observations that are above a given threshold in another survey (e.g. at low extinction), retrieve objects from very large catalogues (e.g. Gaia + WISE) in these non-trivial shapes and not-necessarily-connected regions and combine them to visualise some quantities (e.g. colour-colour diagram). 

## Before we get started
The original tutorial was using Aladin (https://aladin.u-strasbg.fr/) and TOPCAT (http://www.star.bris.ac.uk/~mbt/topcat/), in this version of the tutorial we will use python packages and Aladin to accomplish the same tasks. The python packages include MOCpy (https://github.com/cds-astro/mocpy), astroquery (https://astroquery.readthedocs.io/en/latest/), pyVO (https://pyvo.readthedocs.io/en/latest/), ipyaladin (https://github.com/cds-astro/ipyaladin) as well as astropy and matplotlib. 

Now let's go!

In [1]:
import warnings 
import glob

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from astropy.table import Table
from astropy.io import fits
from astropy.coordinates import SkyCoord, Angle
import astropy.wcs as astropy_wcs
import astropy.units as u
import reproject
import mocpy
import pyvo
from astroquery.vizier import Vizier

# warnings.filterwarnings('ignore')

## Find all Short-Red images associated to the MASH catalog using the VizieR associated data service
The VizieR service at CDS allows you to look for astronomical catalogues that have been published in the literature. Among these catalogues, some contain data associated to the publications and the tables therein. These data can be browsed and explored through the VizieR associated data service, which is linked to the traditional VizieR table service. In the current example we are looking for images associated with the MASH catalogue of planetary nebulae (Parker et al 2006-2008). The MASH fits files are cut-outs extracted from a larger H$\alpha$ and Short Red survey and can be best described as a set of regions of interstes around planetary nebulae. 

To find VizieR associated data, we use the Table Access Protocol (TAP) with the VizieR endpoint. Through the VizieR TAP endpoint we can search for tables, content of tables and for information on associated data. 

First we search for the MASH catalogue:

In [ ]:
tap_vizier = pyvo.dal.TAPService('http://tapvizier.u-strasbg.fr/TAPVizieR/tap')
mash_catalogues = tap_vizier.search("SELECT  *  FROM tables " + 
                                    "WHERE description LIKE '%MASH%Parker%'").to_table()
mash_catalogues

For this tutorial, we are interested in the tables belonging to the catalogues `V/127A`, this includes tables `V/127A/mash1` and `V/127A/mash2`. To have a look at the content of these tables we do the following:

In [ ]:
mash1_head = tap_vizier.search("SELECT TOP 5 * FROM \"V/127A/mash1\" ").to_table()
mash1_head

As you can see, the last column of this table is called `AssocData` and contains the entry `fits`. If you look at this table on the VizieR web interface, you can download the associated fits file. Within this notebook, we query the `obscor` database to get the URLs to the fits file. Using the `astropy.io.fits` module, we can then open the fits files from their URLs.

In [ ]:
mash_fits = tap_vizier.search("SELECT TOP 5  *  FROM obscore " + 
                              "WHERE obs_collection='V/127A'").to_table()
mash_fits

As you can see, the result from this query proivides us with information of the fits files, which are associated with the MASH catalogues. In particular the column `access_url` provides us with the location of the data. To get the first image we could do:

`image = fits.open(mash_fits['access_url'][0])`

and then work on the image, plot it or save it to our machine. However downloading all the data and working with all the data, takes quite some time. So, for the purpose of this tutorial, **we prepared a subsample of 335 of these Short Red images that will run in a timely manner** but we encourage you to repeat with the full Short Red sample later. **The subsample is available at 
http://astro.u-strasbg.fr/~bot/BochBot.tar.gz**

## Create a MOC of the MASH images
The multi-order coverage (MOC) map of a set of images is a useful represation of the sky coverage of these images. MOCs can represent arbitrary patches on the sky, which do not need to be connected. The union or intersection of two MOCs can be calculated quickly and with small computational effort. Catalogues can be filterd by MOCs. 

Here we can create the MOC of the MASH images with the `MOCPy` module. Please note that the majority of the following cell is concerned with removing keywords from the fits file headers, which would otherwise hamper the MOC creation (because the underlying `astropy.wcs.WCS` module uses is confused by these additional DEPRECATED header keywords):

In [ ]:
mash_file_list = glob.glob('Data/MASH_Sample/*.fits')
for file in mash_file_list:
    print(file)
    m = fits.open(file)
    removeable_keywords = ['CDELT1', 'CDELT2', 'PC001001', 'PC001002',
                           'PC002001', 'PC002002', 'CROTA2', 'XPIXELSZ',
                           'YPIXELSZ']
    for keyword in removeable_keywords:
        try:
            del m[0].header[keyword]
        except KeyError:
            continue
    print(m[0].header['CD1_1'])
    m.writeto(file.split('.fits')[0] + '_mod.fits', overwrite=True)
    m.close()
mash_file_list = glob.glob('Data/MASH_Sample/*_mod.fits')
mash_moc = mocpy.MOC.from_fits_images(mash_file_list, max_norder=15)

In [ ]:
sns.set_style('darkgrid')
fig = plt.figure(figsize=(15, 15))
with mocpy.WCS(fig, fov=2.5 * u.deg,
               center=SkyCoord('21:49:00', '-02:30:00', 
                               unit=(u.hourangle, u.deg), frame='galactic'),
               coordsys="galactic", rotation=Angle(0, u.degree),
               projection="AIT") as wcs:
    ax = fig.add_axes([0.17, 0.17, 0.77, 0.77], projection=wcs)
    mash_moc.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color=mpl.cm.magma(0.4))
    ax.set_xlabel('RA')
    ax.set_ylabel('Dec')
    lon, lat = ax.coords[0], ax.coords[1]
    lon.set_major_formatter('hh:mm:ss')
    lat.set_major_formatter('dd:mm')
    lon.set_ticklabel(exclude_overlapping=True)
    lat.set_ticklabel(exclude_overlapping=True)

This figure only shows a small region on the Sky, but you can see how the MOC has arbitrary shapes and not all regions are connected. 

## Load an archival extinction map and create the MOC of the low extinction regions
Different works (e.g. Schlegel et al. 1998, Schlafly  & Finkbeiner 2011, Green et al. 2015 or Dobashi et al. 2013) have created extinction maps of the sky and these maps are publicly available. Some of these maps are all-sky maps, other have higher resolutions, or come from different methods... Such maps are available in healpix format (among others) on the LAMBDA website or the CADE website. For the purpose of this tutorial, we will download the well known all-sky extinction map from Schlegel et al. from the LAMBDA website, and define as a MOC the low extinction area for which 0<E(B-V)<0.4. 
The map is available from here: https://lambda.gsfc.nasa.gov/data/foregrounds/SFD/lambda_sfd_ebv.fits and we save it to disc. 

In [ ]:
ext_map = fits.open('https://lambda.gsfc.nasa.gov/data/foregrounds/SFD/' + 
                    'lambda_sfd_ebv.fits')
ext_map.writeto('Data/Schlegel_extinction_map.fits')

We are only interested in regions with low extinction. To get a MOC of all regions, where the extinction values from the Schlegel et al. map are in the range from 0 to 0.5mag, we first create a mask. This mask is 1 where we want the pixel to be part of the final MOC and 0 everywhere else. With this mask at hand we will create a MOC from the extinction image. This time we will set the MOC resolution to Order 10 (equivalent to a pixelsize of 3.435 arcmin), which will be enough for the purpose of this tutorial. 

However, before we can move on to the creation of the MOC, we need to deal with the fact that the extinction map is in healpix format. This preprocessing closely follows the isntructions here: https://reproject.readthedocs.io/en/stable/healpix.html

In [4]:
hdu_ext_map = fits.open('Data/Schlegel_extinction_map.fits', mode='update')
hdu_ext_map[1].header['COORDSYS'] = 'Galactic'
hdu_ext_map.flush()

In [14]:
target_header = fits.Header.fromstring("""
NAXIS   =                    2
NAXIS1  =                 6288
NAXIS2  =                 3144
CTYPE1  = 'GLON-MOL'
CRPIX1  =                 3144
CRVAL1  =                  0.0
CDELT1  =              -0.1145
CUNIT1  = 'deg     '
CTYPE2  = 'GLAT-MOL'
CRPIX2  =                 1572
CRVAL2  =                  0.0
CDELT2  =               0.1145
CUNIT2  = 'deg     '
COORDSYS= 'Galactic'
""", sep='\n')
array, footprint = reproject.reproject_from_healpix('Data/Schlegel_extinction_map.fits', 
                                                    target_header, nested=True, hdu_in=1)

In [15]:
prim_hdu = fits.PrimaryHDU(data=array, header=target_header)
prim_hdu.writeto('Data/reprojected_Schlegel_extinction_map.fits', overwrite=True)

In [ ]:
mask = array * 0.0
ind_good_pix = (array < 0.5) & (array >= 0.0)
mask[ind_good_pix] = 1.0
ext_moc = mocpy.MOC.from_image(target_header, 10, mask=mask)

In [ ]:
ext_moc.write('extinction_moc.fits')

## Step 5:  Find out which regions are covered by the MASH short red images and in the low extinction regions defined above


 - Go to the “Coverage” menu and select "Logical Operations...".  Select the two MOC planes (the low extinction regions and the MASH subsample) and compute the intersection of both coverages by clicking "Create".
 - In the main Aladin window, a new MOC has been created. You can visualise how this corresponds to regions that are the intersection of the two previous coverages. Unselect the two previous MOCs to see more clearly the intersection coverage.
 

 > __Again this seems like something that MOCpy should be able to do.__

## Step 6: Query the Gaia Catalogue by MOC
Without the usage of MOC, querying for Gaia sources in the low extinction regions covered by the MASH subsample would be tedious or even impossible. Indeed, one would need to load the whole catalog and make selections which would not be possible given the size of the catalog. Alternatively, one would need to query the catalogs field by field, which would take time and several queries. Instead, here, we will use the power of MOC files to query large catalogs directly in the covered regions only. We will use the coverage of the low extinction and MASH-covered region to query for sources from the Gaia surveys, in this highly non-continuous and non-trivial shape areas.
 - On the left side of the Aladin window, type "Gaia" in the select box. A list of all products (images, catalogs, ...) containing this keyword is then displayed. In the catalogs, select the Gaia Source data in the Gaia DR 2 (Gaia2) folder of the available catalogs. Make sure that the intersection coverage plane is selected and choose "in region or MOC", then click on the "Load" button. The catalog will get queried only in the region defined by the MOC (MASH fields at low extinction).
 - If you zoom to one of the MASH images from our sample and at low extinction, you will see the result of the query: WISE sources in each field. If you select sources or click on one, the information from the catalog of each selected source will be displayed.


 > __ Querying catalogues by MOCs should be something that MOCpy is capable of (at least the new mocserver tool in the cds astroquery fork should be able to do so. __

## Step 7: Cross-match Gaia and WISE sources in all fields
We now want to find sources in the selected region (observed in the MASH regions of interests and at low extinction) that are common to the WISE and Gaia catalogs. To do so, we will perform a cross-match of the Gaia sources loaded previously with the WISE all sky catalog. We could download the WISE catalog in our fields of interests in the same way we did for Gaia,but with Aladin v10 we can also directly do the Xmatch without loading the catalog.
 - At the bottom left of the Aladin window, write "WISE" in the select box. Select the Gaia catalog plane on the right in your stack and click on the "WISE All-Sky Data Release (Cutri+2012)" in the list of catalogs on the left. Then choose "via Match" and "Load".



> __Here the translation to python does not seem as obvious as before but I could imagine to use `astroquery.xmatch` to get the data we want.__

## Step 8: Send X-matched catalogue to TOPCAT and build a colour-colour diagram
 - right-click on the X-matched catalog plane in the main Aladin window and select "Broadcast the selected table to..." then "topcat". Your table is then send to the TOPCAT application.
 - In the TOPCAT window, click on the plane plotting button
 - Select the G Gaia magnitude for the Y axis (phot_g_mean_mag) and add "-W1mag" to compute the Gaia G-WISE W1 color and write down the expression for the W1-W2 WISE colour for the X axis (W1mag-W2mag).

You can now select points or regions in this color-magnitude diagram and visualise their 
spatial distribution in Aladin, or go further in the science analysis of this sample...

 > __ This task should be easy to implement by using `matplotlib.pyplot` and/ or `seaborn`. __